# *THU THẬP DỮ LIỆU BẰNG API*

## Danh sách các thành viên:
    - Nguyễn Gia Hân - 19127134 
    - Nguyễn Khắc Vỹ - 19127637 
    - Ngô Đăng Khoa - 19127444 
    - Huỳnh Cao Nhật Hiếu - 19127399 
## Phân công công việc:
    - API:
        + Nguyễn Khắc Vỹ: Xử lí id của playlists và id của những tracks nằm trong playlists do SoundCloud tạo, viết report.
        + Nguyễn Gia Hân: Xử lí id của tracks trong playlist do users tạo, id của users, và thu thập dữ liệu bằng API.
    - HTML:
        + Ngô Đăng Khoa: Xử lí playlists vs tracks, định dạng lại các cell theo hàm.
        + Huỳnh Cao Nhật Hiếu: Lấy users, xử lí features trong user và viết report.

## Thư viện

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import time
import re
import pandas as pd
import json
import os.path
from os import path

# *Function*

## Hàm Crawl_data_from_discover
- Hàm này sẽ đưa ta đến trang discover của soundcloud để có thể thu thập các link playlist có ở trang này. 
- Hàm nhận vào một **class webdriver** của Selenium và return ra một **list gồm tất cả các link** đã thu thập được.
- Đầu tiên driver sẽ get đến trang https://soundcloud.com/discover rồi sau đó thu thập các nút bấm **forward** để có thể load hết các playlist trên một thanh hiển thị và thu thập page source.
- Cuối cùng sử dụng **Beautiful Soup** để thu thập các href từ tag a.

In [2]:
def Crawl_data_from_discover(driver):
    href = []
    buttons = driver.find_elements_by_class_name('tileGallery__sliderButton')
    for button in buttons:# Use forward button to load all playlist
        if button.text == 'forward':
            while True:
                try:
                    button.click()
                except:
                    break
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    Title = soup.find_all("a", class_="playableTile__heading")
    for bg in set(Title):
        href.append('https://soundcloud.com' + bg.attrs['href'])
    return href


In [3]:
driver = webdriver.Chrome(executable_path=r'/mnt/c/DS/SoundCloud/chromedriver.exe')
driver.get("https://soundcloud.com/discover")
driver.find_element_by_id('onetrust-accept-btn-handler').click() #Accept call cookies
href = Crawl_data_from_discover(driver)

/tmp/ipykernel_213/105420257.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'/mnt/c/DS/SoundCloud/chromedriver.exe')
/tmp/ipykernel_213/105420257.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('onetrust-accept-btn-handler').click() #Accept call cookies
/tmp/ipykernel_213/2095301464.py:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  buttons = driver.find_elements_by_class_name('tileGallery__sliderButton')


In [4]:
href

['https://soundcloud.com/iijeriichoii/sets/chill',
 'https://soundcloud.com/discover/sets/charts-trending:metal:vn',
 'https://soundcloud.com/discover/sets/charts-top:techno:vn',
 'https://soundcloud.com/discover/sets/charts-trending:comedy:vn',
 'https://soundcloud.com/discover/sets/charts-top:folksingersongwriter:vn',
 'https://soundcloud.com/alcides-augusto-948527436/sets/after-party',
 'https://soundcloud.com/discover/sets/charts-top:trance:vn',
 'https://soundcloud.com/mohamed-nageb-3/sets/gym',
 'https://soundcloud.com/discover/sets/charts-trending:house:vn',
 'https://soundcloud.com/fitnuslife/sets/motivated-music-july-2016',
 'https://soundcloud.com/yourparadis/sets/jazz-hip3',
 'https://soundcloud.com/mugatunesofficial/sets/audible-adderall-15',
 'https://soundcloud.com/user-610681392/sets/summer-19',
 'https://soundcloud.com/user-267115463/sets/slowed',
 'https://soundcloud.com/discover/sets/charts-trending:danceedm:vn',
 'https://soundcloud.com/user-692461400/sets/lofi',
 'h

## Hàm scroll_webpage
- Dùng để page down một webpage để có thể load được hết page source
- Hàm nhận vào một **class webdriver** của Selenium và **một biến float để làm thời gian dừng để scroll một page**, mặc định biến này = 1
- Đầu tiên sử dụng method execute một câu lệnh script của Selenium để lấy độ cao của webpage và scroll đến cuối page.

In [5]:
def scroll_webpage(driver):
    SCROLL_PAUSE_TIME = 1
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1
    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i)) 
        time.sleep(SCROLL_PAUSE_TIME) 
        i += 1
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 

    

## Hàm id_from_playlist
- Dùng để crawl id của một playlist.
- Hàm nhận vào **một string là đường dẫn đến playlist cần crawl**, **một webdriver của Selenium** và trả về **dictionary chứa thông tin của track** bên trong playlist 
- Hàm lấy data của playlist bằng cách sử dụng một đoạn script để return ra một mảng có tên là **window.__sc_hydration** tìm thấy được các tag script của page source và trong đó chứa mọi thông tin cần thiết của playlist như **Playlist ID**, **title**, **User tạo** và các **ID của track**,... Ở đây chúng ta chỉ lấy id mà thôi.
- Scroll hết webpage và lấy source của playlist đó để lấy các đường dẫn của **track** để tiếp tục crawl tiếp chúng ở các hàm sau.
- Trong hàm này, chỉ dùng html để lấy những tracks id trong playlist của SoundCloud tạo (Playlist không có id). 

In [6]:
def id_from_playlist(url, driver):
    tracks_id_discover = []
    
    driver.get(url)
    scroll_webpage(driver)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    ids = driver.execute_script('return window.__sc_hydration')
    created_by = url.split('/')[3]
    if created_by == 'discover':
        list_id =  ids[5]['data']['tracks']
        playlist_id = None  
    else:
        playlist_id = ids[6]['data']['id']
        list_id =  ids[6]['data']['tracks']

    if(playlist_id == None):
        for track_id in list_id:
            tracks_id_discover.append(track_id.get('id'))

    return playlist_id, tracks_id_discover


## Xử lí toàn bộ link playlist để lấy id và những tracks nằm trong playlist do SoundCloud tạo

### Duyệt hết tất cả đường link có được ở trang discover của SoundCloud.
### List playlists, tracks dùng để chứa id của playlists và tracks.
### Nhận được id và bỏ vào 2 list là playlists và tracks

In [7]:
playlists, tracks = [], []
for links in href:
    playlist, tracks_list = id_from_playlist(links, driver)
    playlists.append(playlist)
    for track in tracks_list:
        tracks.append(track)

## Thu thập dữ liệu bằng API

In [8]:
sleep_time = 1
users = set()
client_id = 'FjnXkiGFvyaVIYtXadMm9pqIDawoxzUW'

### Sử dụng API lấy dữ liệu của playlist, trong dữ liệu này lấy được user_id và track_id
- Lấy tất cả track_id có trong playlist bỏ vào list tracks, đồng thời lưu id tracks có trong playlist theo dạng chuỗi cách nhau bởi dấu phẩy vào một cột trackIds trong dữ liệu của playlist.
- Thêm user_id vào set users
- Lưu dữ liệu vào file playlist.csv

In [9]:
for playlist_id in playlists:
    if (playlist_id != None):
        url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}?client_id={client_id}'
        r = requests.get(url)
        
        while r.ok == False:
            time.sleep(sleep_time)
            r = requests.get(url)
        
        playlist_API = json.loads(r.text)
        
        users.add(playlist_API['user_id'])
        playlist_API.pop('user')
        tracks_list = playlist_API.pop('tracks')
        
        trackIds = ''
        track_count = playlist_API['track_count']
        
        for i in range(0 , track_count -1):
            tracks.append(tracks_list[i]['id'])
            trackIds += str(tracks_list[i]['id'])
            trackIds +=','
        trackIds += str(tracks_list[track_count-1]['id'])
    
        tracks.append(tracks_list[track_count-1]['id'])
        
        playlist_API['trackIds'] = trackIds
    
        df = pd.json_normalize(playlist_API)
        
        if (path.exists("playlist.csv") == False):
            df.to_csv('playlist.csv', mode='a', sep='\t', index=False, encoding= 'utf-8')
        else:
            df.to_csv('playlist.csv', mode='a', header=False, sep='\t', index=False, encoding= 'utf-8')

In [10]:
pd.set_option("display.max_columns", 32)
data_playlist = pd.read_csv('playlist.csv', sep='\t')
data_playlist

,artwork_url,created_at,description,duration,embeddable_by,genre,id,kind,label_name,last_modified,license,likes_count,managed_by_feeds,permalink,permalink_url,public,purchase_title,purchase_url,release_date,reposts_count,secret_token,sharing,tag_list,title,uri,user_id,set_type,is_album,published_at,display_date,track_count,trackIds
0,https://i1.sndcdn.com/artworks-000246771618-hc...,2016-11-15T06:54:05Z,NaN,8180513,all,Chill,275675917,playlist,NaN,2018-09-21T17:29:09Z,all-rights-reserved,8997,False,chill,https://soundcloud.com/iijeriichoii/sets/chill,True,NaN,NaN,NaN,688,NaN,public,NaN,Chill,https://api.soundcloud.com/playlists/275675917,24439798,NaN,False,2016-11-15T06:54:05Z,2016-11-15T06:54:05Z,37,"208174877,101988938,245946661,1688272,6109158,..."
1,NaN,2017-04-03T14:00:59Z,NaN,4648624,all,Relax,312603422,playlist,NaN,2019-01-22T13:14:39Z,all-rights-reserved,4007,False,after-party,https://soundcloud.com/alcides-augusto-9485274...,True,NaN,NaN,NaN,185,NaN,public,afterparty relax,After Party,https://api.soundcloud.com/playlists/312603422,208456866,NaN,False,2017-04-03T14:00:59Z,2017-04-03T14:00:59Z,17,"131058741,182859485,102700900,230219957,194510..."
2,https://i1.sndcdn.com/artworks-000169949629-ye...,2016-07-03T11:15:41Z,NaN,6741102,all,NaN,238293557,playlist,NaN,2019-09-29T09:58:29Z,all-rights-reserved,6223,False,gym,https://soundcloud.com/mohamed-nageb-3/sets/gym,True,NaN,NaN,NaN,271,NaN,public,gym workout,GYM,https://api.soundcloud.com/playlists/238293557,42878728,NaN,False,2016-07-03T11:15:41Z,2016-07-03T11:15:41Z,11,"273629819,270482836,145839137,110741387,234999..."
3,https://i1.sndcdn.com/artworks-000170566278-o5...,2016-07-07T23:22:55Z,Looking for a little workout motivation? Check...,4123564,all,WorkOut,239481577,playlist,NaN,2016-07-07T23:41:56Z,all-rights-reserved,10900,False,motivated-music-july-2016,https://soundcloud.com/fitnuslife/sets/motivat...,True,NaN,NaN,NaN,440,NaN,public,gym fitness motivation running Edm Dance House,Motivated Music - July 2016,https://api.soundcloud.com/playlists/239481577,163911442,NaN,False,2016-07-07T23:41:56Z,2016-07-07T23:41:56Z,18,"271409580,238162839,269226033,270690977,272594..."
4,https://i1.sndcdn.com/artworks-000551931102-ak...,2018-08-12T17:16:02Z,NaN,4026418,all,Jazz,581141859,playlist,NaN,2019-06-15T01:51:56Z,all-rights-reserved,15945,False,jazz-hip3,https://soundcloud.com/yourparadis/sets/jazz-hip3,True,NaN,NaN,NaN,749,NaN,public,Relax Jazz lofi jazzhiphop,Jazz hip3,https://api.soundcloud.com/playlists/581141859,283347384,NaN,False,2018-08-12T17:22:33Z,2018-08-12T17:22:33Z,26,"339215568,243297554,327992677,305391956,413530..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,https://i1.sndcdn.com/artworks-000584116853-gd...,2019-07-23T16:53:05Z,NaN,2583928,all,Jazz & Blues,831880961,playlist,NaN,2021-06-13T14:16:28Z,all-rights-reserved,37595,False,jazzy-night,https://soundcloud.com/tb3dvuqu1ugs/sets/jazzy...,True,NaN,NaN,NaN,1887,NaN,public,Relax study sleep jazz chill,jazzy night,https://api.soundcloud.com/playlists/831880961,245121594,NaN,False,2019-08-18T12:56:37Z,2019-08-18T12:56:37Z,13,"103766647,339825651,219335350,14595652,1164860..."
207,https://i1.sndcdn.com/artworks-000151142988-rh...,2016-02-04T19:00:30Z,Drift away on the ambient waves of the piano\n...,3789582,all,slow piano,192883600,playlist,NaN,2016-07-06T15:44:24Z,all-rights-reserved,143276,False,ambient-piano,https://soundcloud.com/agamidae/sets/ambient-p...,True,NaN,NaN,NaN,9485,NaN,public,"ambient instrumental classical ""ambient piano""...",Ambient piano,https://api.soundcloud.com/playlists/192883600,17971787,NaN,False,NaN,2016-02-04T19:00:30Z,14,"191576787,237941720,199486709,59702269,2187057..."
208,https://i1.sndcdn.com/artworks-aEYZXXd2624TX5Y...,2019-06-11T01:08:58Z,NaN,10348801,all,R&B & Soul,800939679,playlist,NaN,2021-11-12T15:33:03Z,all-rights-reserved,14350,False,unwind-soul-chill-and-r-b,https://soundc

### Sử dụng API lấy dữ liệu của tracks, trong dữ liệu này lấy được thêm user_id.
- Cột user trong dữ liệu get về được bị thừa dữ liệu nên chỉ lấy user_id. Vậy nên, thay cột user thành user_id.
- Đồng thời thêm user_id vào set users. 
- Lưu dữ liệu vào file track.csv

In [11]:
for track_id in tracks:
    url = f'https://api-v2.soundcloud.com/tracks/{track_id}?client_id={client_id}'
    r = requests.get(url)
        
    while r.ok == False:
        time.sleep(sleep_time)
        r = requests.get(url)   
    track_API = json.loads(r.text)
    
    track_API.pop('media')
    track_API.pop('publisher_metadata')
    track_API.pop('visuals')
    user_inf = track_API.pop('user')
    
    users.add(user_inf['id'])
    track_API['user_id'] = user_inf['id']
    
    df = pd.json_normalize(track_API)
    
    if (path.exists("track.csv") == False):
        df.to_csv('track.csv', mode='a', sep='\t', index=False, encoding= 'utf-8')
    else:
        df.to_csv('track.csv', mode='a', header=False, sep='\t', index=False, encoding= 'utf-8')    

In [12]:
pd.set_option("display.max_columns", 44)
data_track = pd.read_csv('track.csv', sep='\t')
data_track

,artwork_url,caption,commentable,comment_count,created_at,description,downloadable,download_count,duration,full_duration,embeddable_by,genre,has_downloads_left,id,kind,label_name,last_modified,license,likes_count,permalink,permalink_url,playback_count,public,purchase_title,purchase_url,release_date,reposts_count,secret_token,sharing,state,streamable,tag_list,title,track_format,uri,urn,user_id,waveform_url,display_date,station_urn,station_permalink,track_authorization,monetization_model,policy
0,https://i1.sndcdn.com/artworks-dOEBzkP5GwAzgiD...,NaN,True,6.0,2021-10-28T10:53:21Z,"EGTS2021 Tiebreaker song. Players, good luck.\...",False,0.0,372036,372036,all,Electronic,False,1149426313,track,NaN,2021-11-07T17:21:26Z,all-rights-reserved,161.0,ice-conclusio-catastropha,https://soundcloud.com/iceloki/ice-conclusio-c...,1936.0,True,NaN,NaN,NaN,12,NaN,public,finished,True,EGTS2021,Conclusio Catastropha,single-track,https://api.soundcloud.com/tracks/1149426313,soundcloud:tracks:1149426313,2832562,https://wave.sndcdn.com/snWpTibfJhd5_m.json,2021-11-07T17:19:32Z,soundcloud:system-playlists:track-stations:114...,track-stations:1149426313,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,ALLOW
1,https://i1.sndcdn.com/artworks-IcaRrZ0UypdOJSw...,NaN,True,55.0,2021-11-01T02:22:33Z,Download the song: https://geoffplaysguitar.ba...,False,0.0,263210,263210,all,Metal,False,1151626678,track,NaN,2021-11-02T06:56:00Z,all-rights-reserved,653.0,megalovania-in-the-style-of-doom-eternal-from-...,https://soundcloud.com/geoffplaysguitar/megalo...,9528.0,True,NaN,NaN,NaN,68,NaN,public,finished,True,"undertale deltarune ""mick gordon"" doom ""doom e...",Megalovania (in the style of Doom Eternal) fro...,single-track,https://api.soundcloud.com/tracks/1151626678,soundcloud:tracks:1151626678,79546796,https://wave.sndcdn.com/9kiIVuD3eM8S_m.json,2021-11-01T02:22:33Z,soundcloud:system-playlists:track-stations:115...,track-stations:1151626678,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,ALLOW
2,https://i1.sndcdn.com/artworks-6oBkPyxdIHz32QN...,NaN,True,0.0,2021-11-07T21:19:24Z,NaN,False,0.0,139703,139703,all,Metal,False,1155594988,track,NaN,2021-11-07T21:34:07Z,all-rights-reserved,56.0,stay-kid-laroi-metal-cover,https://soundcloud.com/sablemetal/stay-kid-lar...,1197.0,True,NaN,NaN,NaN,2,NaN,public,finished,True,"""Kid Laroi"" ""Justin Bieber"" Metalcore",STAY (THE KID LAROI METAL COVER),single-track,https://api.soundcloud.com/tracks/1155594988,soundcloud:tracks:1155594988,246016879,https://wave.sndcdn.com/tTyVdLcuRCQJ_m.json,2021-11-07T21:19:24Z,soundcloud:system-playlists:track-stations:115...,track-stations:1155594988,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,ALLOW
3,https://i1.sndcdn.com/artworks-9O8S7i2yrL2V3mj...,NaN,True,13.0,2021-11-10T13:00:51Z,if i finish this ill make the intro way more i...,False,0.0,23118,23118,all,NaN,False,1157467549,track,NaN,2021-11-10T13:01:29Z,all-rights-reserved,61.0,neurovaniac-v4,https://soundcloud.com/girlwithabigbutt/neurov...,683.0,True,NaN,NaN,NaN,14,NaN,public,finished,True,NaN,neurovaniac v4?,single-track,https://api.soundcloud.com/tracks/1157467549,soundcloud:tracks:1157467549,584320599,https://wave.sndcdn.com/WpGkt2cFWkUM_m.json,2021-11-10T13:00:51Z,soundcloud:system-playlists:track-stations:115...,track-stations:1157467549,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,NOT_APPLICABLE,ALLOW
4,https://i1.sndcdn.com/artworks-CC6w7DCWgw7AL2X...,NaN,True,0.0,2021-11-08T02:59:15Z,The powerful brutal djent metal with aggressiv...,True,1.0,177267,177267,all,Metal,True,1155715915,track,NaN,2021-11-08T02:59:16Z,all-rights-reserved,4.0,brutal-powerful-metal-background-music,https://soundcloud.com/royaltyfreemusic_alexru...,54.0,True,License,https://1.envato.market/GjOmB6,NaN,0,NaN,public,finished,True,"Metal ""royalty free"" audiojungle background Mu...",Brutal & Powerful Metal - Background Music,single-track,https://api.soundcloud.com/tracks/1155715915,soundcloud:trac

### Sử dụng API lấy dữ liệu users của những id user lấy được ở phía trên.
- Lưu dữ liệu vào file user.csv

In [13]:
for user_id in users:
    url = f'https://api-v2.soundcloud.com/users/{user_id}?client_id={client_id}'
    r = requests.get(url)
    while r.ok == False:
        time.sleep(sleep_time)
        r = requests.get(url)   
    user_API = json.loads(r.text)
    
    user_API.pop('creator_subscriptions')
    user_API.pop('creator_subscription')
    user_API.pop('badges')
    
    df = pd.json_normalize(user_API)
    
    if (path.exists("user.csv") == False):
        df.to_csv('user.csv', mode='a', sep='\t', index=False, encoding= 'utf-8')
    else:
        df.to_csv('user.csv', mode='a', header=False, sep='\t', index=False, encoding= 'utf-8') 

In [14]:
pd.set_option("display.max_columns", 32)
data_user = pd.read_csv('user.csv', sep='\t')
data_user

,avatar_url,city,comments_count,country_code,created_at,description,followers_count,followings_count,first_name,full_name,groups_count,id,kind,last_modified,last_name,likes_count,playlist_likes_count,permalink,permalink_url,playlist_count,reposts_count,track_count,uri,urn,username,verified,station_urn,station_permalink,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking
0,https://i1.sndcdn.com/avatars-2VdOHXNpTLn7Pww2...,chicago,18,US,2012-11-17T16:26:17Z,"Raised on the South Side of Chicago, Vic rose ...",331604,9,Vic,Vic Mensa,0,28508171,user,2021-11-02T00:00:01Z,Mensa,16,1,vicsavemoney,https://soundcloud.com/vicsavemoney,15,NaN,129,https://api.soundcloud.com/users/28508171,soundcloud:users:28508171,Vic Mensa.,True,soundcloud:system-playlists:artist-stations:28...,artist-stations:28508171,soundcloud:users:28508171,True,"[{'urn': 'soundcloud:visuals:146210187', 'entr...",NaN
1,https://i1.sndcdn.com/avatars-1FH0LLD3P5Dkiabh...,Toronto,190,CA,2015-06-14T17:41:19Z,Dream State EP out now on all streaming Platfo...,718,42,Eric,Eric Ryan,0,158105618,user,2021-05-29T20:12:26Z,Ryan,40,2,ericryanpascual,https://soundcloud.com/ericryanpascual,2,NaN,12,https://api.soundcloud.com/users/158105618,soundcloud:users:158105618,Eric Ryan ☀️,True,soundcloud:system-playlists:artist-stations:15...,artist-stations:158105618,soundcloud:users:158105618,True,"[{'urn': 'soundcloud:visuals:103502376', 'entr...",NaN
2,https://a1.sndcdn.com/images/default_avatar_la...,NaN,0,NaN,2019-12-02T11:16:41Z,NaN,110,0,NaN,NaN,0,742883347,user,2019-12-02T11:16:41Z,NaN,0,0,matydeejay-music,https://soundcloud.com/matydeejay-music,1,NaN,141,https://api.soundcloud.com/users/742883347,soundcloud:users:742883347,Maty Deejay,False,NaN,soundcloud:system-playlists:artist-stations:74...,artist-stations:742883347,NaN,NaN,NaN
3,https://i1.sndcdn.com/avatars-yLSykC9ixLmgDfwN...,NaN,8,NL,2017-03-27T18:10:59Z,Calundé is the project of The Netherlands base...,439,30,NaN,NaN,0,297992211,user,2021-09-15T08:44:35Z,NaN,9,2,calunde,https://soundcloud.com/calunde,1,NaN,13,https://api.soundcloud.com/users/297992211,soundcloud:users:297992211,CALUNDÉ,False,soundcloud:system-playlists:artist-stations:29...,artist-stations:297992211,soundcloud:users:297992211,True,"[{'urn': 'soundcloud:visuals:145586145', 'entr...",NaN
4,https://a1.sndcdn.com/images/default_avatar_la...,Seoul,0,KR,2012-02-13T12:41:29Z,NaN,33,0,Young,Young Sim,0,12025877,user,2012-02-13T12:44:04Z,Sim,0,0,dasomii,https://soundcloud.com/dasomii,0,NaN,22,https://api.soundcloud.com/users/12025877,soundcloud:users:12025877,Dasomii,False,NaN,soundcloud:system-playlists:artist-stations:12...,artist-stations:12025877,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6810,https://i1.sndcdn.com/avatars-IGb20NzhIPOeyzeg...,ATL,94,NaN,2015-09-16T22:40:20Z,swag lol.,314048,19,Lil Nas,Lil Nas X,0,174817271,user,2021-09-17T04:01:57Z,X,15,1,secret-service-862007284,https://soundcloud.com/secret-service-862007284,10,NaN,37,https://api.soundcloud.com/users/174817271,soundcloud:users:174817271,Lil Nas X,True,soundcloud:system-playlists:artist-stations:17...,artist-stations:174817271,soundcloud:users:174817271,True,"[{'urn': 'soundcloud:visuals:61557448', 'entry...",NaN
6811,https://i1.sndcdn.com/avatars-000503672100-xtj...,"Philadelphia, PA",56,US,2010-12-16T01:57:15Z,BOOKING: greg@datatouring.com\n\nPlatinum Reco...,15811,551,wes,wes walker,0,2424825,user,2020-11-19T16:37:07Z,walker,332,15,weswalker,https://soundcloud.com/weswalker,4,NaN,32,https://api.soundcloud.com/users/2424825,soundcloud:users:2424825,Wes Walker,True,soundcloud:system-playlists:artist-stations:24...,artist-stations:2424825,soundcloud:users:2424825,True,"[{'urn': 'soundcloud:visuals:53677408', 'entry...",NaN
6812,https://i1.sndcdn.com/avatars-000206800148-bn6...,NaN,4,NaN,2015-06-06T14:26:18Z,NaN,688,12,Edouard,Edouard Dion,0,156794876,user,2019-11-22T18:00:02Z,Dion,28,6,